In [4]:
from collections import namedtuple

In [2]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [5]:
HyperParams = namedtuple('HyperParams', ['batch_size', 'num_classes', 'epochs', 'data_augmentation'])

In [6]:
hparams = HyperParams(
    batch_size=32,
    num_classes=10,
    epochs=200,
    data_augmentation=True
)

In [7]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

170475520/170498071 [============================>.] - ETA: 0s(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [8]:
Y_train = keras.utils.to_categorical(y_train, hparams.num_classes)
Y_test = keras.utils.to_categorical(y_test, hparams.num_classes)

In [11]:
model = Sequential()
model.add(Conv2D(
    32, 
    (3, 3), 
    padding='same', 
    input_shape=X_train.shape[1:],
    activation='relu'
))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(hparams.num_classes, activation='softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [12]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
X_train /= 255
X_test /= 255

In [15]:
# Augment the data
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)
datagen.fit(X_train)

In [ ]:
model.fit_generator(
    datagen.flow(X_train, Y_train, batch_size=hparams.batch_size),
    steps_per_epoch=X_train.shape[0] // hparams.batch_size,
    epochs=hparams.epochs,
    validation_data=(X_test, Y_test)
)

Epoch 1/200
  18/1562 [..............................] - ETA: 408s - loss: 2.3048 - acc: 0.1146